## 1. Carga del dataset integrado

En esta sección cargamos el dataset integrado `Combined_Dataset_Features.csv`, que ya contiene la unión de los tres orígenes (Nazario, Nigerian Fraud y SpamAssassin) y algunas características básicas calculadas en el EDA previo. 

El objetivo es tener un punto de partida único para aplicar limpieza de texto y generar nuevas variables (feature engineering) que usarán los modelos de clasificación.

In [1]:
import pandas as pd
import numpy as np
import re

# ===============================
#   1. CARGAR DATASET INTEGRADO
# ===============================
print("Cargando dataset...")
data = pd.read_csv("Combined_Dataset_Features.csv")
print("Filas cargadas:", len(data))

Cargando dataset...
Filas cargadas: 10706


## 2. Limpieza básica de texto

Antes de generar características, normalizamos el texto de las columnas principales (`sender`, `receiver`, `subject`, `body`):

- Convertimos a `str` para evitar errores con valores nulos.
- Eliminamos espacios múltiples.
- Remplazamos saltos de línea por espacios.
- Hacemos un `strip()` para quitar espacios al inicio y al final.

Esto ayuda a estandarizar el texto y evita ruido innecesario en los análisis posteriores.

In [2]:
# ===============================
#   2. LIMPIEZA DE DATOS
# ===============================

# --- Quitar espacios y estandarizar texto ---
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text)        # espacios múltiples
    text = text.replace("\n", " ")          # saltos de línea
    return text.strip()

print("Limpiando columnas de texto...")
data["sender"]   = data["sender"].apply(clean_text)
data["receiver"] = data["receiver"].apply(clean_text)
data["subject"]  = data["subject"].apply(clean_text)
data["body"]     = data["body"].apply(clean_text)

Limpiando columnas de texto...


## 3. Manejo de valores faltantes

Algunas columnas presentan valores nulos, por lo que definimos reglas simples para imputarlos:

- `sender` → `"unknown_sender"`
- `receiver` → `"unknown_receiver"`
- `date` → `"unknown_date"`
- `subject` → `"no_subject"`
- `body` → cadena vacía `""`

Con esto evitamos errores en los cálculos posteriores y dejamos explícito en los datos cuando la información original no estaba disponible.

In [3]:
# --- Manejo de valores faltantes ---
print("Rellenando valores faltantes...")
cols_to_fill = {
    "sender": "unknown_sender",
    "receiver": "unknown_receiver",
    "date": "unknown_date",
    "subject": "no_subject",
    "body": ""
}
data = data.fillna(cols_to_fill)

Rellenando valores faltantes...


## 4. Feature engineering: longitud y número de palabras

En esta sección generamos características numéricas simples relacionadas con el tamaño del texto:

- `subject_len`: número de caracteres en el asunto.
- `body_len`: número de caracteres en el cuerpo.
- `subject_words`: número de palabras en el asunto.
- `body_words`: número de palabras en el cuerpo.

Estas variables capturan qué tan largo es el correo y pueden ser indicativas de ciertos patrones de spam/phishing.

In [4]:
# ===============================
#   3. FEATURE ENGINEERING
# ===============================

print("Generando nuevas características...")

# Longitud de textos
data["subject_len"] = data["subject"].apply(len)
data["body_len"] = data["body"].apply(len)

# Conteo de palabras
data["subject_words"] = data["subject"].apply(lambda x: len(x.split()))
data["body_words"] = data["body"].apply(lambda x: len(x.split()))

Generando nuevas características...


## 5. Feature engineering: porcentaje de mayúsculas

Algunos correos fraudulentos utilizan muchas letras mayúsculas para llamar la atención (por ejemplo, “URGENTE”, “ACTUALIZA TU CUENTA”). 

Calculamos el porcentaje de caracteres en mayúscula en:

- `subject_upper`: proporción de mayúsculas en el asunto.
- `body_upper`: proporción de mayúsculas en el cuerpo.

Esto nos da una medida de “intensidad visual” del mensaje.

In [5]:
# Porcentaje de mayúsculas
def percent_upper(text):
    if len(text) == 0:
        return 0
    return sum(1 for c in text if c.isupper()) / len(text)

data["subject_upper"] = data["subject"].apply(percent_upper)
data["body_upper"] = data["body"].apply(percent_upper)

## 6. Feature engineering: dígitos, caracteres especiales y señales de phishing

Aquí agregamos varias características relacionadas con patrones comunes en correos de phishing:

- `digits_count`: cuántos dígitos aparecen en el cuerpo (frecuente en montos, cuentas, códigos).
- `special_chars`: número de caracteres especiales (por ejemplo `!`, `@`, `#`, etc.).
- `urgency_score`: contador de palabras clave de urgencia presentes en el cuerpo (`urgent`, `verify`, `update`, `warning`, `suspend`, `password`, `bank`, `security`).
- `email_count`: número de direcciones de correo detectadas en el body (regex).
- `url_count`: número de URLs detectadas con patrón `http/https`.

Estas variables ayudan a capturar la “estructura” típica de un correo potencialmente malicioso.

In [6]:
# Conteo de números
data["digits_count"] = data["body"].apply(lambda x: sum(1 for c in x if c.isdigit()))

# Conteo de caracteres especiales
data["special_chars"] = data["body"].apply(lambda x: len(re.findall(r"[^A-Za-z0-9 ]", x)))

# Detección simple de urgencia (palabras comunes en phishing)
UrgencyWords = ["urgent", "verify", "update", "warning", "suspend", "password", "bank", "security"]
data["urgency_score"] = data["body"].apply(
    lambda text: sum(1 for w in UrgencyWords if w.lower() in text.lower())
)

# Detección de emails en el body
data["email_count"] = data["body"].apply(lambda x: len(re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", x)))

# Detección de URL real
data["url_count"] = data["body"].apply(lambda x: len(re.findall(r"https?://\S+", x)))

## 7. Guardado del dataset listo para modelado

Finalmente, guardamos el dataset enriquecido con todas las nuevas características en el archivo:

- `Cleaned_Featured_Dataset.csv`

Este archivo será la base para los experimentos de modelado en Python, Orange y otras herramientas (clasificación, calibración, interpretabilidad, etc.).

In [7]:
# ===============================
#   4. FORMATO FINAL
# ===============================

print("Guardando dataset final para modelado...")

data.to_csv("Cleaned_Featured_Dataset.csv", index=False)

print("\n====================================")
print("  Limpieza + Feature Engineering COMPLETO")
print("  Archivo generado: Cleaned_Featured_Dataset.csv")
print("  Filas finales:", len(data))
print("====================================")

Guardando dataset final para modelado...

  Limpieza + Feature Engineering COMPLETO
  Archivo generado: Cleaned_Featured_Dataset.csv
  Filas finales: 10706
